#### Simple Gen AI APP Using Langchain

In [ ]:
import os
%pip install python-dotenv
from dotenv import load_dotenv
load_dotenv()

os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")
## Langsmith Tracking
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

In [2]:
## Data Ingestion--From the website we need to scrape the data
%pip install langchain_community
from langchain_community.document_loaders import WebBaseLoader

Note: you may need to restart the kernel to use updated packages.


USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
loader=WebBaseLoader("https://www.nhl.com/")
loader

In [4]:
docs=loader.load()
docs

[Document(metadata={'source': 'https://www.nhl.com/', 'title': 'Official Site of the National Hockey League | NHL.com', 'description': 'The official National Hockey League website including news, rosters, stats, schedules, teams, and video.', 'language': 'en'}, page_content="\n\n\n\n\n\nOfficial Site of the National Hockey League | NHL.com\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n        Skip to Main Content\n    \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nNews\n\n\n\n\nVideo\n\n\n\n\nScores\n\n\n\n\n\n\n\nNavigation Menu\n\n\n\n\n\nSchedule\n\n\n\n\nStats\n\n\n\n\nStandings\n\n\n\n\nPlayers\n\n\n\n\nFantasy\n\n\n\n\nNHL Unites\n\n\n\n\nTeams\n\n\n\n\nShop\n\n\n\n\n\n          Shop\n        \n\n\n\nNHL Shop\n\n\n\n\nAuctions\n\n\n\n\nJerseys\n\n\n\n\nMen\n\n\n\n\nWomen\n\n\n\n\nKids\n\n\n\n\nPhotos.com\n\n\n\n\nNHL Shop - NYC\n\n\n\n\n\n\n\nTickets\n\n\n\n\n\n\n\nEN\n\n\n\n\n\n\n\nEnglish\n\n\nFrançais\n\n\nDeutsch\n\n\nSuomi\n

In [5]:
### Load Data--> Docs-->Divide our Docuemnts into chunks dcouments-->text-->vectors-->Vector Embeddings--->Vector Store DB
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents=text_splitter.split_documents(docs)

In [ ]:
documents

In [7]:
%pip install langchain_openai
from langchain_openai import OpenAIEmbeddings
embeddings=OpenAIEmbeddings()

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
%pip install sentence-transformers
%pip install -qU langchain_community faiss-cpu

embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")  # Free model
vectorstoredb = FAISS.from_documents(documents, embeddings)

In [9]:
vectorstoredb

In [11]:
## Query From a vector db
query="what is 4 nations"
result=vectorstoredb.similarity_search(query)
result[0].page_content

"Tkachuks deliver dream performance for U.S. in win\n\n\n\n\n\n\n\n\n\n\n\nU.S. honors Gaudreau legacy in ‘great way’ after win\n\n\n\n\n\n\n\n\n\n\n\nCrosby, MacKinnon leading by example for Canada\n\n\n\n\n\n\n\n\n\n\n\nTop Stories\n\nFinland mulling changes after loss in 4 Nations opener\nTheodore to miss rest of 4 Nations for Canada\nSweden Quarter-Century Teams selected by NHL.com\nSweden seeks bounce back against rival Finland\n2025 NHL Draft Diary: James Hagens\n4 Nations Face-Off blog: Jake Guentzel\nPakarinen knows both sides of Finland, Sweden rivalry\nCrosby provides memorable day for Make-A-Wish teen\nComplete coverage of 4 Nations Face-Off\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nMust Watch\n\n\n\n\n\n\n\n\n3:14\n\n\nTkachuk brothers have big night\n\nFeb 14, 2025\n\n\n\n\n\n\n\n\n\n2:11\n\n\nUSA opens the 3rd with back-to-back goals\n\nFeb 14, 2025\n\n\n\n\n\n\n\n\n\n5:06\n\n\nUSA at FIN | Recap | 4 Nations Face-Off\n\nFeb 14, 2025\n\n\n\n\n\n\n\n\n\n19:34\n\n\nHear from USA players 

In [12]:
from langchain_openai import ChatOpenAI
llm=ChatOpenAI(model="gpt-4o")

In [13]:
## Retrieval Chain, Document chain

from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt=ChatPromptTemplate.from_template(
    """
Answer the following question based only on the provided context:
<context>
{context}
</context>


"""
)

document_chain=create_stuff_documents_chain(llm,prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n\n\n'), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7fec1d89b1f0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7fec34475300>, root_client=<openai.OpenAI object at 0x7fec46933340>, root_async_client=<openai.AsyncOpenAI object at 0x7fec1d89b250>, model_name='gpt-4o', model_kwargs={}, openai_api_key=SecretStr('**********'))
| StrOutputParser(), kwargs={}, config={'run_name': 'stuff_documents_chain'},

In [ ]:
from langchain_core.documents import Document
document_chain.invoke({
    "input":"What are the 4 nations",
    "context":[Document(page_content=doc.page_content) for doc in docs]
})

However, we want the documents to first come from the retriever we just set up. That way, we can use the retriever to dynamically select the most relevant documents and pass those in for a given question.

In [15]:
### Input--->Retriever--->vectorstoredb
vectorstoredb

In [16]:
retriever=vectorstoredb.as_retriever()
from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,document_chain)


In [17]:
retrieval_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7fec1d8991b0>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n\n\n'), additional_kwargs={})])
            |

In [ ]:
## Get the response form the LLM
response=retrieval_chain.invoke({"input":"what is the 4 nations"})
response['answer']

In [ ]:

response

In [ ]:
response['context']